In [0]:
data = [('c00dac11bde74750b4d207b9c182a85f', 'Alex Thomas', 9),('eb6f2d3426694667ae3e79d6274114a4', 'Chris Gray', 2),('3f7b5b8e835d4e1c8b3e12e964a741f3', 'Emily Johnson', 4),('9a345b079d9f4d3cafb2d4c11d20f8ce', 'Michael Brown', 6),('e0a5f57516024de2a231d09de2cbe9d1', 'Jessica Wilson', 3)]

familydf = spark.createDataFrame(data,["id","name","family_size"])
familydf.show()

countrydata = [('023fd23615bd4ff4b2ae0a13ed7efec9', 'Bolivia', 2 , 4),('be247f73de0f4b2d810367cb26941fb9', 'Cook Islands', 4,8),('3e85ab80a6f84ef3b9068b21dbcc54b3', 'Brazil', 4,7),('e571e164152c4f7c8413e2734f67b146', 'Australia', 5,9),('f35a7bb7d44342f7a8a42a53115294a8', 'Canada', 3,5),('a1b5a4b5fc5f46f891d9040566a78f27', 'Japan', 10,12)]

countrydf = spark.createDataFrame(countrydata,["id","name","min_size","max_size"])
countrydf.show()



+--------------------+--------------+-----------+
|                  id|          name|family_size|
+--------------------+--------------+-----------+
|c00dac11bde74750b...|   Alex Thomas|          9|
|eb6f2d3426694667a...|    Chris Gray|          2|
|3f7b5b8e835d4e1c8...| Emily Johnson|          4|
|9a345b079d9f4d3ca...| Michael Brown|          6|
|e0a5f57516024de2a...|Jessica Wilson|          3|
+--------------------+--------------+-----------+

+--------------------+------------+--------+--------+
|                  id|        name|min_size|max_size|
+--------------------+------------+--------+--------+
|023fd23615bd4ff4b...|     Bolivia|       2|       4|
|be247f73de0f4b2d8...|Cook Islands|       4|       8|
|3e85ab80a6f84ef3b...|      Brazil|       4|       7|
|e571e164152c4f7c8...|   Australia|       5|       9|
|f35a7bb7d44342f7a...|      Canada|       3|       5|
|a1b5a4b5fc5f46f89...|       Japan|      10|      12|
+--------------------+------------+--------+--------+



In [0]:
joindf = familydf.join(countrydf, (familydf["family_size"]>=countrydf["min_size"]) & (familydf["family_size"]<=countrydf["max_size"]),"inner").select(familydf["name"],familydf["family_size"],countrydf["name"],"min_size","max_size")
joindf.show()

+--------------+-----------+------------+--------+--------+
|          name|family_size|        name|min_size|max_size|
+--------------+-----------+------------+--------+--------+
|   Alex Thomas|          9|   Australia|       5|       9|
|    Chris Gray|          2|     Bolivia|       2|       4|
| Emily Johnson|          4|     Bolivia|       2|       4|
| Emily Johnson|          4|Cook Islands|       4|       8|
| Emily Johnson|          4|      Brazil|       4|       7|
| Emily Johnson|          4|      Canada|       3|       5|
| Michael Brown|          6|Cook Islands|       4|       8|
| Michael Brown|          6|      Brazil|       4|       7|
| Michael Brown|          6|   Australia|       5|       9|
|Jessica Wilson|          3|     Bolivia|       2|       4|
|Jessica Wilson|          3|      Canada|       3|       5|
+--------------+-----------+------------+--------+--------+



In [0]:
from pyspark.sql.functions import *

groupdf = joindf.groupBy(familydf["name"]).agg(count("*").alias("number_of_countries"))
groupdf.show()

+--------------+-------------------+
|          name|number_of_countries|
+--------------+-------------------+
|   Alex Thomas|                  1|
|    Chris Gray|                  1|
| Emily Johnson|                  4|
| Michael Brown|                  3|
|Jessica Wilson|                  2|
+--------------+-------------------+



In [0]:
finaldf = groupdf.agg(expr("max(number)").alias("number_of_countries"))
finaldf.show()


+-------------------+
|number_of_countries|
+-------------------+
|                  4|
+-------------------+



In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *

#another way 
wn = Window.orderBy(desc("number_of_countries"))

rankdf = groupdf.withColumn("rank",row_number().over(wn))
rankdf.show()

finaldf2 = rankdf.filter(col("rank")==1).drop("rank")
finaldf2.show()

+--------------+-------------------+----+
|          name|number_of_countries|rank|
+--------------+-------------------+----+
| Emily Johnson|                  4|   1|
| Michael Brown|                  3|   2|
|Jessica Wilson|                  2|   3|
|   Alex Thomas|                  1|   4|
|    Chris Gray|                  1|   5|
+--------------+-------------------+----+

+-------------+-------------------+
|         name|number_of_countries|
+-------------+-------------------+
|Emily Johnson|                  4|
+-------------+-------------------+

